In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stat
import statsmodels.api as sm
import statsmodels.formula.api as smf#, ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import ttest_ind, f_oneway
import scipy.stats as stats
import sys
from pathlib import Path
# # Add src directory to the Python path
sys.path.append(str(Path.cwd().parent))
from src.data_analysis.df_dataattribute_analysis import categorize_columns
from src.data_sourcing.import_export_data import get_data, save_data
from src.data_analysis.df_ttest_anova_analysis import run_one_sample_ttest, run_univariate_tests
from src.data_analysis.df_multicollinearity_analysis import compute_vif
from src.data_analysis.df_correlation_analysis import compute_correlations

In [25]:
df = get_data('featured_engineered_testing.csv', 'processed_data')
df.head(5)

2025-01-26 18:26:38,248 - INFO - Loading data from: ..\data\processed\featured_engineered_testing.csv


,ATL_OR_DR,CAMPAIGN_TYPE,CHANNEL,COST,FREE_TRIALS,HOLIDAY_FLAG,MONTH,ATL_OR_DR_CAMPAIGN_TYPE,ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL,ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH,...,LOG_COST,LOG_FREE_TRIALS,LOG_COST_PER_FREE_TRIALS,LOG_COST_BY_FREE_TRIALS,AVG_COST_MONTH,MEDIAN_COST_MONTH,AVG_COST_CAMPAIGN_TYPE_CHANNEL,MEDIAN_COST_CAMPAIGN_TYPE_CHANNEL,AVG_FREE_TRIALS_MONTH,MEDIAN_FREE_TRIALS_MONTH
0,DR - Direct Response,Title,paid social,7784.31,86401.15,False,June,DR - Direct Response_Title,DR - Direct Response_Title_paid social,DR - Direct Response_Title_paid social_June,...,8.959994,11.366768,0.086265,20.326622,16887.563506,3643.265,14781.334567,3309.50,42865.334723,25054.485
1,DR - Direct Response,Title,app,2474.31,2956.74,False,August,DR - Direct Response_Title,DR - Direct Response_Title_app,DR - Direct Response_Title_app_August,...,7.814121,7.992181,0.608045,15.805560,14172.931604,2955.060,15454.451540,3477.08,43953.768382,28127.390
2,ATL - Above The Line,Title,paid social,10222.82,12513.08,False,July,ATL - Above The Line_Title,ATL - Above The Line_Title_paid social,ATL - Above The Line_Title_paid social_July,...,9.232476,9.434610,0.597171,18.666908,13667.902515,2994.470,14781.334567,3309.50,45959.687637,27091.210
3,DR - Direct Response,Title,app,49631.87,17207.04,True,December,DR - Direct Response_Title,DR - Direct Response_Title_app,DR - Direct Response_Title_app_December,...,10.812409,9.753132,1.356967,20.565462,18937.983302,3240.025,15454.451540,3477.08,47495.480279,28640.060
4,ATL - Above The Line,Title,paid social,2081.06,21758.33,False,August,ATL - Above The Line_Title,ATL - Above The Line_Title_paid social,ATL - Above The Line_Title_paid social_August,...,7.641113,9.987798,0.091343,17.628385,14172.931604,2955.060,14781.334567,3309.50,43953.768382,28127.390


### Multi-collinearity Testing Analysis

In [77]:
# df.columns

# X_df= df.drop(columns=['FREE_TRIALS', 'ATL_OR_DR_CAMPAIGN_TYPE', 'ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL', 'ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH',
#                          'ATL_OR_DR_CAMPAIGN_TYPE_MONTH', 'ATL_OR_DR_CHANNEL_MONTH', 'CAMPAIGN_TYPE_CHANNEL_MONTH', 'CAMPAIGN_TYPE_MONTH', 'CHANNEL_MONTH',
#                            'LOG_FREE_TRIALS', 'ATL_OR_DR_MONTH'], axis=1)
# X_df.columns
#
hashmap = categorize_columns(df)
formula = hashmap['numerical']
formula.remove('FREE_TRIALS')
formula.remove('LOG_FREE_TRIALS')
formula.remove('LOG_COST')
formula.extend(['CHANNEL'])
print("formula:", formula)
temp_df = df[formula]
vif_result = compute_vif(
    data=temp_df,
)
vif_result
# '''
# Try an aggregation calculation by Channel and month? So 2 group by's?
# '''

formula: ['COST', 'COST_PER_FREE_TRIALS', 'COST_BY_FREE_TRIALS', 'LOG_COST_PER_FREE_TRIALS', 'LOG_COST_BY_FREE_TRIALS', 'AVG_COST_MONTH', 'MEDIAN_COST_MONTH', 'AVG_COST_CAMPAIGN_TYPE_CHANNEL', 'MEDIAN_COST_CAMPAIGN_TYPE_CHANNEL', 'AVG_FREE_TRIALS_MONTH', 'MEDIAN_FREE_TRIALS_MONTH', 'CHANNEL']


,Feature,VIF
0,MEDIAN_FREE_TRIALS_MONTH,5.880656
1,AVG_COST_MONTH,4.546466
2,MEDIAN_COST_MONTH,4.018094
3,COST,3.649319
4,AVG_FREE_TRIALS_MONTH,3.060894
5,CHANNEL_paid social,2.750514
6,CHANNEL_youtube,2.422387
7,COST_BY_FREE_TRIALS,2.165662
8,CHANNEL_display,2.028226
9,COST_PER_FREE_TRIALS,1.973539


### One Sample T-Test Aanalysis

In [63]:
categorical_list = ['ATL_OR_DR','HOLIDAY_FLAG']
quantitative = ["FREE_TRIALS", "LOG_FREE_TRIALS"]

# 2) Using a dictionary for different hypothesized means per column:
results_2 = run_one_sample_ttest(
    df=df,
    categorical_cols=categorical_list,
    quantitative_cols=quantitative,
    popmean={"FREE_TRIALS": 277769, "LOG_FREE_TRIALS": 10.23},  # different popmeans
    stat_type="median",
    alpha=0.05,  # more stringent significance level
    sort_results=True
)
results_2

,Categorical Variable,Category,Quantitative Variable,Sample Size,Sample Stat,T-statistic,p-value,Decision,Error
0,ATL_OR_DR,DR - Direct Response,FREE_TRIALS,5507,27833.420000,-294.961263,0.000000,Reject H0,None
1,ATL_OR_DR,ATL - Above The Line,FREE_TRIALS,8601,27769.030000,-349.593644,0.000000,Reject H0,None
2,HOLIDAY_FLAG,False,FREE_TRIALS,12629,27775.900000,-430.288018,0.000000,Reject H0,None
3,HOLIDAY_FLAG,True,FREE_TRIALS,1479,27789.190000,-153.775053,0.000000,Reject H0,None
4,HOLIDAY_FLAG,True,LOG_FREE_TRIALS,1479,10.232438,-0.796542,0.425845,Fail to Reject H0,None
5,ATL_OR_DR,ATL - Above The Line,LOG_FREE_TRIALS,8601,10.231713,-0.178557,0.858290,Fail to Reject H0,None
6,ATL_OR_DR,DR - Direct Response,LOG_FREE_TRIALS,5507,10.234029,-0.155741,0.876243,Fail to Reject H0,None
7,HOLIDAY_FLAG,False,LOG_FREE_TRIALS,12629,10.231960,0.026885,0.978552,Fail to Reject H0,None


In [62]:
# 2) Using a dictionary for different hypothesized means per column:
results_2 = run_one_sample_ttest(
    df=df,
    categorical_cols=categorical_list,
    quantitative_cols=quantitative,
    popmean={"FREE_TRIALS": 45458, "LOG_FREE_TRIALS": 10.22},  # different popmeans
    stat_type="mean",
    alpha=0.05,  # more stringent significance level
    sort_results=True
)
results_2

,Categorical Variable,Category,Quantitative Variable,Sample Size,Sample Stat,T-statistic,p-value,Decision,Error
0,HOLIDAY_FLAG,False,LOG_FREE_TRIALS,12629,10.230238,1.154430,0.248346,Fail to Reject H0,None
1,ATL_OR_DR,ATL - Above The Line,LOG_FREE_TRIALS,8601,10.228078,0.750651,0.452883,Fail to Reject H0,None
2,ATL_OR_DR,DR - Direct Response,LOG_FREE_TRIALS,5507,10.227902,0.586744,0.557399,Fail to Reject H0,None
3,HOLIDAY_FLAG,True,LOG_FREE_TRIALS,1479,10.208979,-0.417617,0.676288,Fail to Reject H0,None
4,HOLIDAY_FLAG,True,FREE_TRIALS,1479,44986.705767,-0.311335,0.755590,Fail to Reject H0,None
5,ATL_OR_DR,DR - Direct Response,FREE_TRIALS,5507,45547.216388,0.113320,0.909781,Fail to Reject H0,None
6,HOLIDAY_FLAG,False,FREE_TRIALS,12629,45513.573500,0.102958,0.917998,Fail to Reject H0,None
7,ATL_OR_DR,ATL - Above The Line,FREE_TRIALS,8601,45401.434358,-0.085102,0.932182,Fail to Reject H0,None


### ANOVA Testing Analysis

In [56]:
hashmap = categorize_columns(df)
categorical = hashmap['categorical']
categorical.remove('ATL_OR_DR')
categorical.remove('HOLIDAY_FLAG')
categorical

['CAMPAIGN_TYPE',
 'CHANNEL',
 'MONTH',
 'ATL_OR_DR_CAMPAIGN_TYPE',
 'ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL',
 'ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH',
 'ATL_OR_DR_CAMPAIGN_TYPE_MONTH',
 'ATL_OR_DR_CHANNEL',
 'ATL_OR_DR_CHANNEL_MONTH',
 'ATL_OR_DR_MONTH',
 'CAMPAIGN_TYPE_CHANNEL',
 'CAMPAIGN_TYPE_CHANNEL_MONTH',
 'CAMPAIGN_TYPE_MONTH',
 'CHANNEL_MONTH']

In [87]:
quantitative = ["FREE_TRIALS", "LOG_FREE_TRIALS"]

anova_results_df = run_univariate_tests(
    df=df,
    categorical_cols=categorical,
    quantitative_cols=quantitative,
    test_type="anova"
)
anova_results_df[anova_results_df['p-value'] <= 0.50]

,Categorical Variable,Quantitative Variable,F-statistic,p-value,Error
0,MONTH,FREE_TRIALS,1.745376,0.057730,None
1,ATL_OR_DR_MONTH,FREE_TRIALS,1.437204,0.080345,None
2,CAMPAIGN_TYPE_MONTH,FREE_TRIALS,1.346628,0.104410,None
3,CHANNEL_MONTH,FREE_TRIALS,1.096844,0.170871,None
4,MONTH,LOG_FREE_TRIALS,1.335306,0.197357,None
5,CAMPAIGN_TYPE_CHANNEL_MONTH,FREE_TRIALS,1.061845,0.206772,None
6,ATL_OR_DR_CAMPAIGN_TYPE_MONTH,FREE_TRIALS,1.112372,0.268880,None
7,ATL_OR_DR_CHANNEL_MONTH,FREE_TRIALS,1.050398,0.285439,None
8,ATL_OR_DR_MONTH,LOG_FREE_TRIALS,1.085444,0.352263,None
9,ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH,FREE_TRIALS,1.023266,0.361860,None


### Kruskal Wallis Testing Analysis

In [89]:
kruskal_results_df = run_univariate_tests(
    df=df,
    categorical_cols=categorical,
    quantitative_cols=quantitative,
    test_type="kruskal"
)
kruskal_results_df[kruskal_results_df['p-value'] <= 0.60]

,Categorical Variable,Quantitative Variable,H-statistic,p-value,Error
0,MONTH,FREE_TRIALS,15.421565,0.163999,None
1,MONTH,LOG_FREE_TRIALS,15.421565,0.163999,None
2,ATL_OR_DR_MONTH,LOG_FREE_TRIALS,26.406948,0.282034,None
3,ATL_OR_DR_MONTH,FREE_TRIALS,26.406948,0.282034,None
4,CHANNEL_MONTH,LOG_FREE_TRIALS,193.675803,0.493063,None
5,CHANNEL_MONTH,FREE_TRIALS,193.675803,0.493063,None
6,ATL_OR_DR_CHANNEL_MONTH,LOG_FREE_TRIALS,233.572700,0.586903,None
7,ATL_OR_DR_CHANNEL_MONTH,FREE_TRIALS,233.572700,0.586903,None


### Pearson Correlation Testing Analysis

In [94]:
# 1) Pearson correlation (numeric only - only linear relationships)
pearson_results = compute_correlations(
    df=df,
    include_categorical=False,
    cat_cols=None  # Do not encode categorical variables
)
pearson_results[
    (pearson_results['p-value'] < 0.5) &
    (pearson_results['Variable 1'] == 'FREE_TRIALS')
]

,Variable 1,Variable 2,Correlation,p-value,Method,Error
4,FREE_TRIALS,LOG_COST_BY_FREE_TRIALS,0.347763,0.000000e+00,pearson,None
8,FREE_TRIALS,LOG_FREE_TRIALS,0.747446,0.000000e+00,pearson,None
21,FREE_TRIALS,COST_BY_FREE_TRIALS,0.219831,5.792956e-154,pearson,None
22,FREE_TRIALS,LOG_COST_PER_FREE_TRIALS,-0.215401,9.361971e-148,pearson,None
33,FREE_TRIALS,COST_PER_FREE_TRIALS,-0.062575,1.013470e-13,pearson,None
39,FREE_TRIALS,AVG_FREE_TRIALS_MONTH,0.036881,1.177749e-05,pearson,None
51,FREE_TRIALS,MEDIAN_FREE_TRIALS_MONTH,0.024740,3.295094e-03,pearson,None
76,FREE_TRIALS,AVG_COST_MONTH,0.007530,3.711244e-01,pearson,None


### Spearman Correlation Testing Analysis

In [95]:
# 2) Spearman correlation (with categorical one-hot encoding)
spearman_results = compute_correlations(
    df=df,
    include_categorical=True,  # Encode categorical variables
    drop_first=True,
    cat_cols=categorical
)
spearman_results[(spearman_results['Variable 1'] == 'FREE_TRIALS') & (spearman_results['p-value'] < 0.5)]

,Variable 1,Variable 2,Correlation,p-value,Method,Error
8,FREE_TRIALS,LOG_COST_BY_FREE_TRIALS,0.470630,0.000000,spearman,None
9,FREE_TRIALS,LOG_COST_PER_FREE_TRIALS,-0.460507,0.000000,spearman,None
10,FREE_TRIALS,LOG_FREE_TRIALS,1.000000,0.000000,spearman,None
12,FREE_TRIALS,COST_PER_FREE_TRIALS,-0.460507,0.000000,spearman,None
13,FREE_TRIALS,COST_BY_FREE_TRIALS,0.470630,0.000000,spearman,None
42,FREE_TRIALS,MEDIAN_FREE_TRIALS_MONTH,0.031010,0.000230,spearman,None
49,FREE_TRIALS,AVG_FREE_TRIALS_MONTH,0.023191,0.005875,spearman,None
66,FREE_TRIALS,AVG_COST_MONTH,0.015087,0.073137,spearman,None
68,FREE_TRIALS,MEDIAN_COST_CAMPAIGN_TYPE_CHANNEL,-0.010339,0.219440,spearman,None
78,FREE_TRIALS,LOG_COST,0.006565,0.435562,spearman,None


### Random Testing Analysis

In [12]:
# import pandas as pd
# import statsmodels.api as sm
# from statsmodels.formula.api import ols

# # Example: Assuming your data is in a DataFrame called df
# # Replace 'FREE_TRIALS' with your dependent variable
# # Replace other variable names with the actual column names from your dataset

# # Create the OLS formula
# formula = """
# FREE_TRIALS ~ HOLIDAY_FLAG
# + ATL_OR_DR + CAMPAIGN_TYPE + CHANNEL + MONTH
# + ATL_OR_DR_CAMPAIGN_TYPE + ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL
# + ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH + ATL_OR_DR_CAMPAIGN_TYPE_MONTH
# + ATL_OR_DR_CHANNEL + ATL_OR_DR_CHANNEL_MONTH + ATL_OR_DR_MONTH
# + CAMPAIGN_TYPE_CHANNEL + CAMPAIGN_TYPE_CHANNEL_MONTH + CAMPAIGN_TYPE_MONTH + CHANNEL_MONTH
# + COST + COST_PER_FREE_TRIALS + COST_BY_FREE_TRIALS
# + LOG_COST + LOG_COST_PER_FREE_TRIALS + LOG_COST_BY_FREE_TRIALS
# + AVG_COST_MONTH + MEDIAN_COST_MONTH
# + AVG_COST_CAMPAIGN_TYPE_CHANNEL + MEDIAN_COST_CAMPAIGN_TYPE_CHANNEL
# + AVG_FREE_TRIALS_MONTH + MEDIAN_FREE_TRIALS_MONTH
# """
# # formula = """
# # FREE_TRIALS ~ HOLIDAY_FLAG
# # + MONTH + CHANNEL + CAMPAIGN_TYPE + ATL_OR_DR
# # + COST + COST_PER_FREE_TRIALS + LOG_COST
# # + MONTH:CHANNEL + CAMPAIGN_TYPE:ATL_OR_DR
# # """
# #formula = "FREE_TRIALS ~ MONTH * ATL_OR_DR + CHANNEL"

# # Fit the OLS model
# # model = ols(formula, data=df).fit()

# import statsmodels.api as sm
# model = sm.GLM(df['FREE_TRIALS'], sm.add_constant(X), family=sm.families.Poisson()).fit()
# print(model.summary())


# # Print the summary of the regression model
# print(model.summary())


In [13]:
# pd.DataFrame(model.summary())

In [14]:
# # # Define a function to plot boxplots and detect outliers based on IQR
# # def plot_outliers(column, ax):
# #     ax.boxplot(data[column], vert=False, patch_artist=True, boxprops=dict(facecolor="lightblue"))
# #     ax.set_title(f'Boxplot of {column}')
# #     ax.set_xlabel(column)

# # # Create boxplots for COST and FREE_TRIALS
# # fig, axes = plt.subplots(1, 2, figsize=(12, 5))
# # plot_outliers('COST', axes[0])
# # plot_outliers('FREE_TRIALS', axes[1])
# # plt.tight_layout()
# # plt.show()

# # Calculate outlier thresholds using IQR
# def calculate_outliers(column):
#     q1 = data[column].quantile(0.25)
#     q3 = data[column].quantile(0.75)
#     iqr = q3 - q1
#     lower_bound = q1 - 1.5 * iqr
#     upper_bound = q3 + 1.5 * iqr
#     outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
#     return lower_bound, upper_bound, outliers

# # Remove outliers based on IQR thresholds for LOG_COST and LOG_FREE_TRIALS
# log_cost_bounds = calculate_outliers('LOG_COST')
# log_free_trials_bounds = calculate_outliers('LOG_FREE_TRIALS')

# # Filter data to exclude outliers in LOG_COST and LOG_FREE_TRIALS
# filtered_log_data = data[
#     (data['LOG_COST'] >= log_cost_bounds[0]) & (data['LOG_COST'] <= log_cost_bounds[1]) &
#     (data['LOG_FREE_TRIALS'] >= log_free_trials_bounds[0]) & (data['LOG_FREE_TRIALS'] <= log_free_trials_bounds[1])
# ]

# # Display the size of the dataset before and after outlier removal based on log variables
# log_filtered_size = filtered_log_data.shape[0]

# log_filtered_size


In [15]:
# from sklearn.preprocessing import StandardScaler

# # Select numerical columns to standardize
# numerical_features = ['LOG_COST', 'LOG_COST_SQ']
# scaler = StandardScaler()

# # Standardize numerical features
# filtered_log_data_encoded[numerical_features] = scaler.fit_transform(
#     filtered_log_data_encoded[numerical_features]
# )

# # Re-run OLS regression with standardized features
# X_standardized = filtered_log_data_encoded[['LOG_COST', 'LOG_COST_SQ'] +
#                                            [col for col in filtered_log_data_encoded.columns
#                                             if col.startswith(('ATL_OR_DR_', 'CAMPAIGN_TYPE_', 'CHANNEL_'))]]
# X_standardized = sm.add_constant(X_standardized)  # Add constant for OLS

# y_standardized = filtered_log_data_encoded['LOG_FREE_TRIALS']

# # Fit OLS regression with standardized features
# ols_standardized_model = sm.OLS(y_standardized, X_standardized).fit()

# # Display the updated summary
# ols_standardized_summary = ols_standardized_model.summary()
# ols_standardized_summary